# Median of 2 sorted lists

## Setup

### Imports

In [1]:
from pydantic import BaseModel
import numpy as np
import pandas as pd

### Utils

In [2]:
def pad(x):
    return [-float("inf")] + x + [float("inf")]

class Pointers():
    def __init__(self, high, target, low):
        self.high = high
        self.target = target
        self.low = low

class BorderValues():
    def __init__(self, max_low, min_high):
        self.max_low = max_low
        self.min_high = min_high

class OrderedList:
    def __init__(self, nums, target=None):
        self.list = nums
        self.padded_list = pad(nums)
        self.n = len(self.padded_list)
        self.search_idx = Pointers(low=0, target=self.n // 2, high=self.n)
        # override target if given
        if target != None:
            self.set_target(target)
        else:
            self.set_border_values()

    def set_target(self, target):
        self.search_idx.target = target
        self.set_border_values()
        
    def set_border_values(self):
        self.split = BorderValues(
            max_low=self.padded_list[self.search_idx.target - 1], 
            min_high=self.padded_list[self.search_idx.target],
        )
        
    def next_target(self, up=True):
        if up:
            self.search_idx.low = self.search_idx.target
        else:
            self.search_idx.high = self.search_idx.target
        self.search_idx.target = (self.search_idx.low + self.search_idx.high) // 2
        self.set_border_values()

    def __repr__(self):
        return str(self.list)
        
def find_median(a, b, target_index):
    a_lower_lte_b_upper = a.split.max_low <= b.split.min_high
    b_lower_lte_a_upper = b.split.max_low <= a.split.min_high
    
    if not a_lower_lte_b_upper:
        a.next_target(up=False)
        b.search_idx.target = target_index - a.search_idx.target
        b.set_border_values()
        a, b = find_median(a, b, target_index)
    elif not b_lower_lte_a_upper:
        a.next_target(up=True)
        b.search_idx.target = target_index - a.search_idx.target
        b.set_border_values()
        a, b = find_median(a, b, target_index)
        
    return a, b
    
def calculate_median(a, b):
    n = a.n + b.n
    max_low = max(a.split.max_low, b.split.max_low)
    min_high = min(a.split.min_high, b.split.min_high)
    #print(median_input_values)
    if n % 2 == 0:
        med = (max_low + min_high) / 2.0
    else:
        med = min_high
        
    return med
    
def main(nums1, nums2):
    a, b = OrderedList(nums1), OrderedList(nums2)
    # make sure a contains the shorter list
    a, b = (a, b) if a.n <= b.n else (b, a)
    target_index = (a.n + b.n) // 2
    b.set_target(target_index - a.search_idx.target)
    a, b = find_median(a, b, target_index)
    med = calculate_median(a, b)
    return a, b, med

In [3]:
def highlight_ol(ol: OrderedList):
    green_cols = list(range(ol.search_idx.low, ol.search_idx.target))
    blue_cols = list(range(ol.search_idx.target, ol.search_idx.high))
    s = (
        pd.Series(ol.padded_list).to_frame().T
        .style
        .set_properties(subset=green_cols, **{"background-color": "lightgreen"})
        .set_properties(subset=blue_cols, **{"background-color": "lightblue"})
    )
    return s

In [4]:
override_a = False
override_b = False

na_ = np.random.randint(1, 100)
nb_ = np.random.randint(1, 100)

a_ = np.random.randint(0, 60, na_).tolist()
a_.sort()

# override a
#override_a = True
if override_a:
    a_ = [1]
    na_ = len(a_)

b_ = np.random.randint(40, 100, nb_).tolist()
b_.sort()

# override b
#override_b = True
if override_b:
    b_ = [1]
    nb_ = len(b_)

c_ = np.concat([a_, b_]).astype(int).tolist()
c_.sort()
n_ = len(c_)
med_calc = np.median(c_)

display(highlight_ol(OrderedList(c_)))
print(f"calculated median = {med_calc}")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85
0,-inf,2.000000,4.000000,9.000000,9.000000,10.000000,11.000000,11.000000,12.000000,12.000000,17.000000,17.000000,18.000000,18.000000,19.000000,20.000000,22.000000,23.000000,30.000000,32.000000,32.000000,32.000000,33.000000,36.000000,38.000000,38.000000,39.000000,40.000000,40.000000,41.000000,42.000000,42.000000,42.000000,43.000000,43.000000,43.000000,43.000000,44.000000,44.000000,44.000000,45.000000,45.000000,46.000000,47.000000,49.000000,50.000000,52.000000,53.000000,53.000000,54.000000,54.000000,54.000000,55.000000,55.000000,57.000000,57.000000,58.000000,58.000000,58.000000,58.000000,60.000000,60.000000,61.000000,67.000000,67.000000,69.000000,70.000000,71.000000,74.000000,74.000000,75.000000,75.000000,76.000000,77.000000,77.000000,78.000000,78.000000,82.000000,82.000000,84.000000,86.000000,88.000000,90.000000,91.000000,96.000000,inf


calculated median = 46.5


In [5]:
a, b, med = main(a_, b_)
try:
    display(highlight_ol(a))
    display(highlight_ol(b))
except:
    pass
finally:
    display(med)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43
0,-inf,2.000000,4.000000,9.000000,9.000000,10.000000,11.000000,11.000000,12.000000,12.000000,17.000000,17.000000,18.000000,18.000000,19.000000,20.000000,22.000000,23.000000,30.000000,32.000000,32.000000,32.000000,33.000000,36.000000,38.000000,38.000000,39.000000,43.000000,43.000000,43.000000,44.000000,44.000000,45.000000,47.000000,52.000000,53.000000,53.000000,54.000000,54.000000,54.000000,55.000000,55.000000,57.000000,inf


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43
0,-inf,40.000000,40.000000,41.000000,42.000000,42.000000,42.000000,43.000000,44.000000,45.000000,46.000000,49.000000,50.000000,57.000000,58.000000,58.000000,58.000000,58.000000,60.000000,60.000000,61.000000,67.000000,67.000000,69.000000,70.000000,71.000000,74.000000,74.000000,75.000000,75.000000,76.000000,77.000000,77.000000,78.000000,78.000000,82.000000,82.000000,84.000000,86.000000,88.000000,90.000000,91.000000,96.000000,inf


46.5

In [6]:
print(f"calculated med == med: {med_calc == med}")

calculated med == med: True
